### Summary

We would like to take an evolutionary MSA for DHFR and create an energy function following [2013Cocco](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003176). Then we will use this energy function to score single and double mutants from the Homo sapiens version of DHFR. 

The energy function takes the form of a probability model [Eqn (7) 2013Cocco](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003176#pcbi.1003176.e059)
$$ P(a_1, \ldots, a_L) = \frac{1}{Z} \exp{ \bigg\{ \frac{1}{2} \sum_{i,j} e_{ij}(a_i, a_j) + \sum_{i} h_i(a_i) \bigg\} },$$
 which we estimate from the MSA. The probability represents the probability of seeing a given sequence $(a_1, \ldots, a_L)$ in the MSA. Using the mean field approximation we get values for the coupling matrix $e_{ij}(a_i,a_j)$ and the local fields $h_i(a_i)$. It is hard to calculate the normalization constant $Z$ as $L$ is large. However, since $Z$ is a constant for every sequence in this MSA we can ignore it and get a relative score for each sequence by estimating $\log (PZ)$.
 
**Sections**
1. [Import pre-computed data and generate covariance matrix](#gencovmat)
2. [Generate probability model parameters](#probparams)
3. [Calculate relative score for human DHFR](#relativescore)
4. [Create Mutants Energy file](#createmutantsenergy)

### Import pre-computed data and generate covariance matrix <a id="gencovmat" />

In [1]:
import os
import numpy as np
import scipy
import scipy.linalg

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
AMINO_ACIDS = np.array([aa for aa in "RKDEQNHSTCYWAILMFVPG-"], "S1")

In [3]:
# read in the first line of the alignment (which has the human version of DHFR)
datadir = "../data"
msa_file = os.path.join(datadir, "DHFR.aln")
with open(msa_file) as fh:
    human_DHFR = np.array([[x for x in fh.readline().strip()] ], np.dtype("S1"))

human_DHFR.tostring().decode('ascii')

'VRPLNCIVAVSQNMGIGKNGDLPWPPLRNEFKYFQRMTTTSSVEGKQNLVIMGRKTWFSIPEKNRPLKDRINIVLSRELKEPPRGAHFLAKSLDDALRLIEQPELASKVDMVWIVGGSSVYQEAMNQPGHLRLFVTRIMQEFESDTFFPEIDLGKYKLLPEYPGVLSEVQEEKGIKYKFEVYEKKD'

In [4]:

weights_file = os.path.join(datadir, "DHFR.weights.npy")

print("Loading weights from : ", weights_file)
weights = np.load(weights_file)

single_site_marginal_file = os.path.join(datadir, "DHFR.single.npy")
double_site_marginal_file = os.path.join(datadir, "DHFR.double.npy")

print("Loading single site marginals from ", single_site_marginal_file)
f_i_a = np.load(single_site_marginal_file)

print("Loading double site marginals from ", double_site_marginal_file)    
f_i_j_a_b = np.load(double_site_marginal_file)

Loading weights from :  ../data/DHFR.weights.npy
Loading single site marginals from  ../data/DHFR.single.npy
Loading double site marginals from  ../data/DHFR.double.npy


In [5]:
# Get the length of the sequence and the length of the alphabet from the imported matrices
L, q = f_i_a.shape
M_eff = sum(weights) # number of effected sequences

In [6]:
# Add pseudo count
pseudo_count_ratio = 0.5

f_i_a = (pseudo_count_ratio / q ) + (1 - pseudo_count_ratio) * f_i_a / M_eff
f_i_j_a_b = (pseudo_count_ratio  / (q*q) ) + (1 - pseudo_count_ratio) * f_i_j_a_b / M_eff

# The formula for f_i_j_a_b is a little different when i==j
# essentially we have f_i_a on the diagonal and zero's everywhere else. 
for i in range(L):
    f_i_j_a_b[i, :, i, :] = np.diag(f_i_a[i, :])

In [7]:
# Covariance Matrix
# We take an outer product of f_i_a with itself using numpy's broadcasting rules. 
# This gives us a matrix where the (i,a, j, b) index is f[i,a] * f[j,b]
C_i_j_a_b = f_i_j_a_b  - f_i_a[:, :, np.newaxis, np.newaxis] * f_i_a[np.newaxis, np.newaxis, :, :] 

# we project the covariance matrix down the first q-1 elements
# Since the frequencies add up to 1 we can discard amino-acid value (a = q) for each site
# without losing any information
C_i_j_a_b = C_i_j_a_b[:, :(q-1), :, :(q-1)]
print("C_i_j_a_b.shape = {}".format(C_i_j_a_b.shape)) 

C_i_j_a_b.shape = (186, 20, 186, 20)


### Generate probability model parameters <a id="probparams" />

From [Eqn (8) 2013Cocco](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003176#pcbi.1003176.e071) we have 
$$e_{ij}(a,b) = (C^{-1})_{ij}(a,b) \qquad \forall i,j, \enspace \forall a,b=1, \ldots, q-1$$ and $$e_{ij}(a,q) = e_{ij}(q,a) = 0 \qquad \forall a = 1, \ldots, q,$$ and $$ h_i(q) = 0.$$

In [8]:
cov = C_i_j_a_b.reshape((L*(q-1), L*(q-1)))
invCov = np.linalg.inv(cov)
e_i_j_a_b =  - invCov.reshape((L, q-1, L, q-1))

Now that we have the coupling matrix $e_{ij}(a_i, a_j)$, we need to calculate the local fields $h_i(a_i)$. To do this we use [Eqn (S6) from the Supporting Information of 2013Cocco](https://doi.org/10.1371/journal.pcbi.1003176.s001). 

$$h_i(a) = \log \bigg( \frac{f_i(a)}{f_i(q)} \bigg) - \frac{1}{L} \sum_{\mu j b} \xi_i^\mu(a) \xi_j^\mu (b) f_j(b)$$

Here we are not distinguishing between positive and negative patterns to lighten the notations. From [Eqn 18 2013 Cocco](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003176#pcbi.1003176.e144) we have $$ e_{ij}(a,b) = \frac{1}{L} \sum_\mu \xi_i^\mu (a) \xi_j^\mu (b).$$ This gives us
$$ \sum_{jb} e_{ij}(a,b) f_j(b) = \frac{1}{L} \sum_{\mu j b} \xi_i^\mu (a) \xi_j^\mu (b) f_j(b).$$ Substituting this back into Eqn (S6) we get an expression for the local fields in terms of the coupling matrix and the mariginal frequencies. 
$$ h_i(a) = \log \bigg( \frac{f_i(a)}{f_i(q)} \bigg) - \sum_{jb} e_{ij}(a,b) f_j(b).$$ Here, $b$ runs from $1$ to $q-1$.


In [9]:
h_i_a = np.log(f_i_a[:, :(q-1)] / f_i_a[:, q-1, np.newaxis]) - \
    (e_i_j_a_b * f_i_a[np.newaxis, np.newaxis, :, :(q-1)]).sum(axis=(2,3))

h_i_a.shape

(186, 20)

### Calculate relative score for human DHFR  <a id="relativescore" />

In [10]:
human_DHFR_idx = np.zeros(human_DHFR.shape, dtype=np.int)
for i, a in enumerate(AMINO_ACIDS):
    human_DHFR_idx[human_DHFR == a] = i
human_DHFR_idx = human_DHFR_idx.squeeze()
    
if human_DHFR.tostring().decode('ascii').find("-") >= 0 :
    raise ValueError("We should not have gaps in this protein")

# the array below tells us the index of the amino acid in each position
human_DHFR_idx[:10]

array([17,  0, 18, 14,  5,  9, 13, 17, 12, 17])

In [11]:
# seq_runner is just an array from 0 to the length of the alignment that we can use to index
seq_runner = np.array(range(L), np.int)
def calculate_energy_function(idx):
    """ idx is a 1-D numpy array of length equal to L and 
        each value is the index of amino acid i in the AMINO_ACIDS array.
        This function uses seq_runner, e_i_j_a_b and h_i_a arrays. 
    """
    # meshgrid creates a "tuple" product of the indices by repeating the index
    # along the x axis in xv and repeating it along the yaxis in yv
    xv, yv = np.meshgrid(idx, idx)
    energy = np.sum(e_i_j_a_b[seq_runner, xv, seq_runner, yv ]) / 2. + \
        h_i_a[seq_runner, idx].sum()
    return(energy)

In [12]:
human_DHFR_energy = calculate_energy_function(human_DHFR_idx)
human_DHFR_energy

-588711.3156041729

### Create Mutants Energy file <a id="createmutantsenergy" />

*We can speed this up by only calculating the energy difference between mutant sequences and WT*

In [13]:
output_dir = os.path.join(datadir, "output", "mutagenesis")
!mkdir -p $output_dir

In [14]:
# single mutants
single_mutant_energy_filename = os.path.join(output_dir, "single_mutants.tsv")
single_mutant_energy_filename

'../data/output/mutagenesis/single_mutants.tsv'

In [15]:
with open(single_mutant_energy_filename, "w") as fh:
    AA_ascii = AMINO_ACIDS.tostring().decode('ascii')
    for i in seq_runner:
        aa_idx = human_DHFR_idx[i]
        aa = AA_ascii[aa_idx]
        for mut_idx in range(1, (q-1)): # do not switch to the gap symbol
            if (mut_idx != aa_idx): # do not switch to mutant to the same symbol as WT
                mut = AA_ascii[mut_idx]
                mut_DHFR_idx = human_DHFR_idx.copy()
                mut_DHFR_idx[i] = mut_idx
                energy = calculate_energy_function(mut_DHFR_idx)
                print("{0:}{1:}{2:}\t{3:}".format(aa, i + 1, mut, energy), file=fh)

In [16]:
# inspect and compress the file
!head $single_mutant_energy_filename
!gzip -f $single_mutant_energy_filename

V1K	-588798.0856454724
V1D	-588517.8028304778
V1E	-588795.4418900218
V1Q	-588542.1029375127
V1N	-588530.3745272686
V1H	-588341.319892712
V1S	-588652.157661544
V1T	-588476.5636452194
V1C	-588293.1471080026
V1Y	-588463.4439595203


In [17]:
# double mutants
double_mutant_energy_filename = os.path.join(output_dir, "double_mutants.tsv")
double_mutant_energy_filename

'../data/output/mutagenesis/double_mutants.tsv'

In [18]:
progress_bar = True
try:
    from IPython.display import clear_output
except ImportError:
     progress_bar = False

In [23]:
with open(double_mutant_energy_filename, "w") as fh:
    AA_ascii = AMINO_ACIDS.tostring().decode('ascii')
    for i in seq_runner:
        aa1_idx = human_DHFR_idx[i]
        aa1 = AA_ascii[aa1_idx]
        for j in seq_runner:
            if (i < j): #if i = j then we are creating single mutants
                if progress_bar:
                    clear_output(wait=True)
                print ("Processing site i=", i+1, ",j=", j+1 , " of ", L)
                aa2_idx = human_DHFR_idx[j]
                aa2 = AA_ascii[aa2_idx]
                for mut1_idx in range(1, (q-1)): # do not switch to the gap symbol
                    mut1 = AA_ascii[mut1_idx]
                    for mut2_idx in range(1, (q-1)):
                        if mut1_idx != mut2_idx:
                            mut2 = AA_ascii[mut2_idx]
                            mut_DHFR_idx = human_DHFR_idx.copy()
                            mut_DHFR_idx[i] = mut1_idx
                            mut_DHFR_idx[j] = mut2_idx
                            energy = calculate_energy_function(mut_DHFR_idx)
                            print("{}{}{},{}{}{}\t{}".format(aa1, i+1, mut1, aa2, j+1, mut2, energy), file=fh)

Processing site i= 185 ,j= 186  of  186


In [22]:
# inspect and compress the file
!head $double_mutant_energy_filename
!gzip -f $double_mutant_energy_filename

V1K,R2D	-588743.1948797524
V1K,R2E	-589020.8339392963
V1K,R2Q	-588767.4949867873
V1K,R2N	-588755.7665765432
V1K,R2H	-588566.7119419866
V1K,R2S	-588877.5497108187
V1K,R2T	-588701.955694494
V1K,R2C	-588518.5391572772
V1K,R2Y	-588688.836008795
V1K,R2W	-588566.9674016556
